# Module 3.2 Add-on: `circuit.parameters` shows only unassigned symbols  

In [1]:

# Metadata
import sys, platform
from importlib.metadata import version, PackageNotFoundError
def safe_ver(pkg):
    try:
        return version(pkg)
    except PackageNotFoundError:
        return "not installed"
print("Python:", sys.version.split()[0])
print("Qiskit:", safe_ver("qiskit"))
print("qiskit-ibm-runtime:", safe_ver("qiskit-ibm-runtime"))


Python: 3.12.9
Qiskit: 2.2.0
qiskit-ibm-runtime: 0.41.1


In [2]:

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke

backend = FakeSherbrooke()



## Demonstration  

- We create a circuit using only a subset of symbols from a `ParameterVector`.  
`circuit.parameters` lists the **unassigned** symbols still present. After  
`assign_parameters(..., inplace=True)`, the set becomes empty if all used  
symbols were bound.
- We will also show that you cannot assign parameters that are not used in the circuit.

In [5]:

theta = ParameterVector('theta', 3)  # theta[0], theta[1], theta[2]
phi = ParameterVector('phi', 2)      # not used here

cir = QuantumCircuit(2, 2)
cir.h(0)
cir.rx(theta[0], 0)   # uses theta[0]
cir.ry(theta[1], 1)   # uses theta[1]

print("Before assignment, cir.parameters should show theta[0] and theta[1]:")
print(sorted(p.name for p in cir.parameters))

# ---- Uncomment to show error ----
# Try to assign all entries of 'theta' using vector-as-key mapping 
# cir.assign_parameters({theta: [0.1, 0.1, 0.1]}, inplace=True)  

cir.assign_parameters({theta[0]: 0.1, theta[1]:0.1}, inplace=True)  

print("\nAfter in-place assignment, cir.parameters should be empty:")
print(sorted(p.name for p in cir.parameters))


Before assignment, cir.parameters should show theta[0] and theta[1]:
['theta[0]', 'theta[1]']

After in-place assignment, cir.parameters should be empty:
[]


Notice that even though we did not use `theta[2]` it doesn't appear when we print   
`cir.parameters` because only parameters that are used in the circuit are printed.


## Optional: compile and evaluate with `EstimatorV2`  

We compile with a preset pass manager targeting a fake backend and run one shot  
of `EstimatorV2` to confirm everything is concrete (no remaining symbols).  

In [ ]:

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
cir_isa = pm.run(cir)

obs = SparsePauliOp('XX')
obs_isa = obs.apply_layout(cir_isa.layout)

pub = (cir_isa, obs_isa, None, 0.003)
estimator = EstimatorV2(mode=backend)
job = estimator.run([pub], shots=1024, precision=0.5)
res = job.result()
print("Estimator run complete. Metadata keys:", list(res[0].metadata.keys()))
